<a href="https://colab.research.google.com/github/oh-be/Deep-Learning/blob/main/NNPredictionOptimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing Neural Network

## Preprocessing

### Imports

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
url = 'https://raw.githubusercontent.com/oh-be/Deep-Learning/main/Resources/charity_data.csv'
application_df = pd.read_csv(url)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Pipelines

Original

In [ ]:
def default_pipeline(application_df):
    
    # Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
    application_df = application_df.drop(labels=["EIN", "NAME"], axis=1)

    # Bucket APPLICATION_TYPE
    application_counts = application_df["APPLICATION_TYPE"].value_counts()

    # Determine which values to replace if counts are less than 500
    replace_application = list(application_counts[application_counts < 500].index)

    # Replace in dataframe
    for app in replace_application:
        application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

    # Bucket CLASSIFICATION
    classification_counts = application_df["CLASSIFICATION"].value_counts()

    # Determine which values to replace if counts are less than 1800
    replace_class = list(classification_counts[classification_counts < 1800].index)

    # Replace in dataframe
    for cls in replace_class:
        application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")

    # Generate our categorical variable lists
    application_cat = list(application_df.dtypes[application_df.dtypes == "object"].index)

    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

    # Add the encoded variable names to the dataframe
    encode_df.columns = enc.get_feature_names(application_cat)

    # Merge one-hot encoded features and drop the originals
    application_df = application_df.merge(encode_df, left_index=True, right_index=True)
    application_df = application_df.drop(labels=application_cat, axis=1)
    return application_df    

Reduced Bins

In [ ]:
def reduced_pipeline(application_df):
    
    # Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
    application_df = application_df.drop(labels=["EIN", "NAME"], axis=1)

    # Bucket APPLICATION_TYPE
    application_counts = application_df["APPLICATION_TYPE"].value_counts()

    # Determine which values to replace if counts are less than 500
    replace_application = list(application_counts[application_counts < 500].index)

    # Replace in dataframe
    for app in replace_application:
        application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

    # Bucket CLASSIFICATION
    classification_counts = application_df["CLASSIFICATION"].value_counts()

    # Determine which values to replace if counts are less than 1800
    replace_class = list(classification_counts[classification_counts < 1800].index)

    # Replace in dataframe
    for cls in replace_class:
        application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")
        
    income_counts = application_df["INCOME_AMT"].value_counts()
        
    # Determine which values to replace if counts are less than 3000
    replace_income = list(income_counts[income_counts < 3000].index)

    # Replace in dataframe
    for income in replace_income:
        application_df["INCOME_AMT"] = application_df["INCOME_AMT"].replace(income, "Other")
    
    aff_counts = application_df["AFFILIATION"].value_counts()
    
    # Determine which values to replace if counts are less than 15000
    replace_aff = list(aff_counts[aff_counts < 15000].index)

    # Replace in dataframe
    for aff in replace_aff:
        application_df["AFFILIATION"] = application_df["AFFILIATION"].replace(aff, "Other")

    # Generate our categorical variable lists
    application_cat = list(application_df.dtypes[application_df.dtypes == "object"].index)

    # Create a OneHotEncoder instance
    enc = OneHotEncoder(sparse=False)

    # Fit and transform the OneHotEncoder using the categorical variable list
    encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

    # Add the encoded variable names to the dataframe
    encode_df.columns = enc.get_feature_names(application_cat)

    # Merge one-hot encoded features and drop the originals
    application_df = application_df.merge(encode_df, left_index=True, right_index=True)
    application_df = application_df.drop(labels=application_cat, axis=1)
    
    # Drop "SPECIAL_CONSIDERATION_N":
    application_df = application_df.drop("SPECIAL_CONSIDERATIONS_N", axis=1)
    return application_df    

In [ ]:
# instantiating df's for pipelines above
df = default_pipeline(application_df)
df2 = reduced_pipeline(application_df)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Reset dataset function (_for new testing_)

In [ ]:
def new_dataset(df):
    # Split our preprocessed data into our features and target arrays
    y = df["IS_SUCCESSFUL"].values.reshape(-1, 1)
    X = df.drop("IS_SUCCESSFUL", axis=1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train = X_scaler.transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = new_dataset(df)

## Compile, Train and Evaluate the Models

### define models for quick boot

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# number of layer1 neurons = 2*(number of inputs=43) = 86 ~ 80
hidden_nodes_layer1 = 80

# number of layer2 neurons: Between (input=80) and (output=1 - classifier)
hidden_nodes_layer2 = 30

tanh = tf.keras.models.Sequential()

# First hidden layer
tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
tanh.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
tanh.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 80)                3520      
                                                                 
 dense_22 (Dense)            (None, 30)                2430      
                                                                 
 dense_23 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


# relu-sig model with variations

In [27]:
X_train, X_test, y_train, y_test = new_dataset(df)
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# number of layer1 neurons = 2*(number of inputs=43) = 86 ~ 80
hidden_nodes_layer1 = 80

# number of layer2 neurons: Between (input=80) and (output=1 - classifier)
hidden_nodes_layer2 = 30

sig = tf.keras.models.Sequential()

# First hidden layer
sig.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
sig.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
sig.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
sig.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 80)                3520      
                                                                 
 dense_25 (Dense)            (None, 30)                2430      
                                                                 
 dense_26 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
sig.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model_sig = sig.fit(X_train, y_train, epochs=95)

Epoch 1/95
804/804 [==============================] - 2s 2ms/step - loss: 0.5762 - accuracy: 0.7183
Epoch 2/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5603 - accuracy: 0.7252
Epoch 3/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5556 - accuracy: 0.7282
Epoch 4/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5537 - accuracy: 0.7286
Epoch 5/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7293
Epoch 6/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5511 - accuracy: 0.7306
Epoch 7/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5506 - accuracy: 0.7304
Epoch 8/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5500 - accuracy: 0.7292
Epoch 9/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7294
Epoch 10/95
804/804 [==============================] - 1s 2ms/step - loss: 0.5484 - accuracy: 0.7319

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = sig.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5555 - accuracy: 0.7367 - 626ms/epoch - 2ms/step
Loss: 0.5554803609848022, Accuracy: 0.7366763949394226


In [28]:
X_train, X_test, y_train, y_test = new_dataset(df)

In [29]:
# Compile the model
tanh.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model_tanh = tanh.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5671 - accuracy: 0.7219
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5550 - accuracy: 0.7293
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5508 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5490 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5487 - accuracy: 0.7310
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5475 - accuracy: 0.7348
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7312
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5453 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = tanh.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5566 - accuracy: 0.7287 - 404ms/epoch - 2ms/step
Loss: 0.5565647482872009, Accuracy: 0.7287463545799255


In [31]:
# Export and save model to HDF5 file
tanh.save("Optimization_original_tanh.h5")
sig.save("Optimization_original_relu.h5")

In [32]:
# Using the refined dataset

In [36]:
X_train, X_test, y_train, y_test = new_dataset(df2)
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# number of layer1 neurons = 2*(number of inputs=43) = 86 ~ 80
hidden_nodes_layer1 = 80

# number of layer2 neurons: Between (input=80) and (output=1 - classifier)
hidden_nodes_layer2 = 50

sig = tf.keras.models.Sequential()

# First hidden layer
sig.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
sig.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
sig.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
sig.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 80)                2800      
                                                                 
 dense_28 (Dense)            (None, 50)                4050      
                                                                 
 dense_29 (Dense)            (None, 1)                 51        
                                                                 
Total params: 6,901
Trainable params: 6,901
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Compile the model
sig.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model_sig = sig.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5761 - accuracy: 0.7149
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5647 - accuracy: 0.7210
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5614 - accuracy: 0.7241
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5595 - accuracy: 0.7238
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5581 - accuracy: 0.7242
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5571 - accuracy: 0.7262
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5565 - accuracy: 0.7261
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5558 - accuracy: 0.7266
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5553 - accuracy: 0.7266
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5553 - accura

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = sig.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5438 - accuracy: 0.7420 - 379ms/epoch - 1ms/step
Loss: 0.5438106656074524, Accuracy: 0.7420408129692078


In [40]:
X_train, X_test, y_train, y_test = new_dataset(df2)

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# number of layer1 neurons = 2*(number of inputs=43) = 86 ~ 80
hidden_nodes_layer1 = 80

# number of layer2 neurons: Between (input=80) and (output=1 - classifier)
hidden_nodes_layer2 = 50

tanh = tf.keras.models.Sequential()

# First hidden layer
tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
tanh.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
tanh.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
tanh.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_33 (Dense)            (None, 80)                2800      
                                                                 
 dense_34 (Dense)            (None, 50)                4050      
                                                                 
 dense_35 (Dense)            (None, 1)                 51        
                                                                 
Total params: 6,901
Trainable params: 6,901
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Compile the model
tanh.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model_tanh = tanh.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5775 - accuracy: 0.7133
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5632 - accuracy: 0.7231
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5611 - accuracy: 0.7241
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5595 - accuracy: 0.7247
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5578 - accuracy: 0.7249
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5569 - accuracy: 0.7255
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5556 - accuracy: 0.7280
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5549 - accuracy: 0.7264
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5548 - accuracy: 0.7261
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5535 - accura

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = tanh.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5437 - accuracy: 0.7373 - 366ms/epoch - 1ms/step
Loss: 0.5437089204788208, Accuracy: 0.7372594475746155


In [43]:
# Export and save model to HDF5 file
tanh.save("Optimization_refined_tanh.h5")
sig.save("Optimization_refined_relu.h5")